<a href="https://colab.research.google.com/github/francji1/01ZLMA/blob/main/R/01ZLMA_ex03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01ZLMA - Exercise 03
Exercise 03 of the course 01ZLMA. 

## Contents

* Statistical Inference
 ---
* Testing
 ---
* ..



#  Necessary theory recap from Lecture 04

Under the conditions of regularity holds

1.  $ \ U(\beta) \sim N_{p}(0,I(\beta)) \Rightarrow  I^{-\frac{1}{2}}(\beta)\, U(\beta) {\stackrel{D}{\longrightarrow}} N_{p}(0, 1)$
2. $ U(\beta)I^{-1}(\beta)U(\beta)\sim \chi^{2}(p) \Rightarrow U(\beta)^T I^{-1}(\beta)U(\beta)  {\stackrel{D}{\longrightarrow}} \chi^{2}(p)$
3. Consistency of $\hat{\beta}$ and Wald statistics: \\
 $\hat{\beta}\sim N_{p}(\beta,I^{-1}(\beta)) \Rightarrow
(\hat{\beta}-\beta)^T I(\beta)(\hat{\beta}-\beta) {\stackrel{D}{\longrightarrow}} \chi^{2}(p)$



Saturated and null model

* Null model: $\mu_i = \mu, \forall i$ \\
The Null Model assumes one parameter for all of the data points, which means you only estimate 1 parameter. 
* Saturated model: $Y_i = \hat{\mu_i}, \forall i$ \\
The Saturated Model is a model that assumes each data point has its own parameters, which means you have n parameters to estimate.

Questions:
* What is the difference between null and saturated model?
* Which model has greater log-likelihoood value?
* Which model has the highest log-likelihood value?
* What can you say about asymptotic distributions of $\hat{\beta}$ and $U(\hat{\beta})$ for saturated model?

## Let's code 

In [ ]:
library(tidyverse)
library(lubridate)
library(MASS)

#For sure: set dplyr functions
select    <- dplyr::select;
rename    <- dplyr::rename;
mutate    <- dplyr::mutate; 
summarize <- dplyr::summarize;
arrange   <- dplyr::arrange;
slice     <- dplyr::slice;
filter    <- dplyr::filter;
recode    <- dplyr::recode

Use Example 2 from Exercise 02

In [ ]:
n  = 20
m  = 2
X1 = rep(rep(1,n),m)
X2 = rep(seq(1,n,1),m)
X  = cbind(X1,log(X2))
beta = c(0.9, 1.3)
lamdas = exp(X%*%beta) # Means
Y  = rpois(n*m,lamdas)


In [ ]:
model <- glm(formula=Y~log(X2),family=poisson(link = "log"))
summary(model) # souhrn modelu
beta.e <- coefficients(model); beta.e  # estimated parameters
y.hat  <- model$fitted.values

#############################################
## Plot data and estimation
plot(X2,Y, col="red", cex=1.5, lwd=2, 
      main="Poisson model",
      xlab="Year Quoter",
      ylab="Number of cases",
      cex.lab=1.2)
lines(unique(y.hat), col="blue", type = "l", lwd=3)
text <- c("model", "data")
legend(x=1, y=160, text, col = c(4,2), bty="n", lwd = 2,  cex=1.3, lty=c(1,0))
dev.off()

In [ ]:
# function to calcualate weights
calc_W_inv <- function(X,beta){
  n = length(X[,1])
  W = diag(c(exp(X%*%beta)),n,n)
  return(W)
}

In [ ]:
# function to calcualate weights
calc_Z <- function(X,Y,beta){
  Z = X%*%(beta) + (Y-exp(X%*%beta))/exp(X%*%beta)
  return(Z)
 }

In [ ]:
# IWLS for example 2
IWLS <- function(X,Y,beta_init,maxiter,epsilon){
  result <- list(FM=NA, SV=NA, betas=NA)
  # Fisher-scoring algorithm
  i <- 1     # first iteration
  convergence <- F
  beta_i <- beta_init


  while (convergence == F & i <= maxiter){
    W = calc_W_inv(X,beta_i)
    Z = calc_Z(X,Y,beta_i)
    beta_pred = beta_i
    beta_i = solve(t(X)%*%W%*%X, t(X)%*%W%*%Z)
    diff   = max(abs(beta_i - beta_pred))
    if (diff>=epsilon){
      i <- i+1
    }
    else
      convergence <- T
    }
    W    <- calc_W_inv(X,beta_i)
    Z = calc_Z(X,Y,beta_i)
    result$SV <- t(X)%*%W%*%Z
    result$FM <- t(X)%*%W%*%X
    result$betas <- solve(t(X)%*%W%*%X, t(X)%*%W%*%Z)

  return(result)
}

In [ ]:
# Estimation of betas
result1 <- IWLS(X,Y,c(1,1),100,10^(-6))
result1$betas      # Estimation of parameters
result1$FM
solve(result1$FM)  # Estimated covariance matrix  = Inverse of estimated Fisher information matrix
result1$SV         # Score Vector

In [ ]:
model <- glm(formula=Y~log(X2),family=poisson(link = "log"))
#s_mod <- summary(model) 
s_mod <- summary.glm(model)  
# the unscaled (dispersion = 1) estimated covariance matrix of the estimated coefficients.
FIM1 <- s_mod$cov.unscaled 
FIM1

Asymptotics:

* $ (\hat{\beta} - \beta) \sim N_{p}(0, I^{-1}(\beta))$ 
* $\hat{I}(\hat{\beta}) = (X^T \hat{W} X)$
* $\hat{V} (\hat{\beta}) = (X^T \hat{W} X)^{-1}$

In [ ]:
n = 10
repet = 50
n_observ = c(1,2,5,10,100)
betas_hat = list(matrix(NA,repet,2),matrix(NA,repet,2),matrix(NA,repet,2),matrix(NA,repet,2),matrix(NA,repet,2))


for (i in 1:length(n_observ)){
  for (j in c(1:repet)){
  X1 = rep(rep(1,n),1)
  X2 = rep(seq(1,n,1),n_observ[i])
  X  = cbind(X1,log(X2))
  beta = c(0.9, 1.3)
  lamdas = exp(X%*%beta) # Means
  Y  = rpois(n*n_observ[i],lamdas)
 # betas_hat[[i]][j,] <- IWLS(X,Y,c(1,1),1000,10^(-8))$betas
  betas_hat[[i]][j,] <- glm(formula=Y~log(X2),family=poisson(link = "log"))$coeff
  }
}



In [ ]:
for (i in 1:length(n_observ)){
print(paste("Number of observations:",n_observ[i]*n))
print(cov(betas_hat[[i]] - beta))
print(mean(betas_hat[[i]] - beta))
}

## Hypothesis testing

Use the model from the beginning again.

In [ ]:
n  = 20
m  = 2
X1 = rep(rep(1,n),m)
X2 = rep(seq(1,n,1),m)
X  = cbind(X1,log(X2))
beta = c(0.9, 1.3)
lamdas = exp(X%*%beta) # Means
Y  = rpois(n*m,lamdas)


In [ ]:
model <- glm(formula=Y~log(X2),family=poisson(link = "log"))
#s_mod <- summary(model) 
s_mod <- summary.glm(model)  
summary.glm(model)
# the unscaled (dispersion = 1) estimated covariance matrix of the estimated coefficients.
FIM1 <- s_mod$cov.unscaled 
FIM1

Calculation of z value
 $$Z_i = \frac{\hat{\beta_i}}{(I^{-1}(\hat{\beta_i}))_{ii}}$$

In [ ]:
# Testing statistics from summary table
beta_hat <- s_mod$coefficients;
beta_hat  

# By definition
z_stat = model$coeff/sqrt(diag(s_mod$cov.unscaled))
z_stat
z_stat = beta_hat[,1]/beta_hat[,2];
z_stat

In [ ]:
# p-values of the test
p.val <- 2*pnorm(z_stat, mean = 0, sd = 1, lower.tail = FALSE, log.p = FALSE);
p.val

In [ ]:
### 100(1-alpha) confidence interval
alpha <- 0.05
u <- qnorm(1-alpha/2,0,1)
CI_LB <- beta_hat[,1] - u * beta_hat[,2]
CI_UB <- beta_hat[,1] + u * beta_hat[,2]

cbind("2.5% CI" = CI_LB,ESTIM = beta_hat[,1], "97.5% CI" = CI_UB)


# bulit in function
confint(model, level = 0.95)

Question:

* Compare hypothesis testing in LM vs. GLM

# Deviance


Log-likelihood of the saturated model is the highest possible one with given data, $\tilde{\mu}_i = y_i$ and $\tilde{\theta_i} = \theta(y) = (b')^{-1}(y_i)$.
$$l(\tilde{\mu},\phi;y)=\sum_{i=1}^{n}\frac{y_{i}\tilde{\theta}_{i}-b(\tilde{\theta}_{i})}{a_{i}(\phi)}+\sum_{i=1}^{n}c(y_i,\phi)$$

Scale deviance statistics:
$${S(y,\hat{\mu},\phi)}=2\left[l(\tilde{\mu},\phi;y)-l(\hat{\mu},\phi;y)\right]
=2\sum_{i=1}^{n}\frac{y_{i}(\tilde{\theta}_{i}-\hat{\theta}_{i})
-\left(b(\tilde{\theta}_{i})-b(\hat{\theta}_{i})\right)}{a_{i}(\phi)}.
$$

Deviance:
Let $a_{i}(\phi)=a_{i}\phi$, then
$$S(y,\hat{\mu},\phi)=\frac{D(y,\hat{\mu})}{\phi},
$$
and
$$
D(y,\hat{\mu})=2\sum_{i=1}^{n}\frac{y_{i}(\tilde{\theta}_{i}-\hat{\theta}_{i})
-\left(b(\tilde{\theta}_{i})-b(\hat{\theta}_{i})\right)}{a_{i}}
$$

### Comparison of two models

Assume model $D_0$ with $p_0$ paramters and its sub-model $D_1$ with $p_1$ parameters, then
$$ \frac{1}{\phi} (D_0 - D_1) \sim \chi(p_0 - p_1) $$

Question:
* Can we take deviance as a measure of model quality?
* Complete the sentence: Compare GLMs with deviance is like compare LMs with ...

In [ ]:
# Add random variable to the previous model
###  vlozeni nezavisle promenne 
Z  <- runif(n*m,0,1)
model_0 <- glm(formula=Y~log(X2)+Z,family=poisson(link = "log"))
model_1 <- glm(formula=Y~log(X2),family=poisson(link = "log"))
# null model
model_n <- glm(formula=Y~1,family=poisson(link = "log"))
# saturated model
I <- diag(1,n*m,n*m)
model_s <- glm(formula=Y~I,family=poisson(link = "log"))


summary(model_0)
summary(model_1)
summary(model_n)
summary(model_s)


For Poisson model:
$$D = 2 \sum_{i=1}^n y_i log( \frac{y_i}{\hat{\mu_i}})$$

In [ ]:
mu_est_0 <- model_0$fitted.values
mu_est_1 <- model_1$fitted.values


Dev_0 <- 2*sum(Y*log(Y/mu_est_0))
Dev_0
Dev_1 <- 2*sum(Y*log(Y/mu_est_1))
Dev_1


## Anova testing 
from anova.glm?

The table will optionally contain test statistics (and P values) comparing the reduction in deviance for the row to the residuals. For models with known dispersion (e.g., binomial and Poisson fits) the chi-squared test is most appropriate, and for those with dispersion estimated by moments (e.g., gaussian, quasibinomial and quasipoisson fits) the F test is most appropriate. 

Mallows' Cp statistic is the residual deviance plus twice the estimate of $sigma^2$ times the residual degrees of freedom, which is closely related to AIC (and a multiple of it if the dispersion is known). You can also choose "LRT" and "Rao" for likelihood ratio tests and Rao's efficient score test. The former is synonymous with "Chisq" (although both have an asymptotic chi-square distribution). 

In [ ]:
anova(model_1)
anova(model_1, test = "Cp")
anova(model_1, test = "Chisq")


anova(model_1, model_0, test = "Rao")
anova(model_1,model_0, test = "LRT")   
  

In [ ]:
# p-value of deviance tst
# H0: model fit data
p_dev <- pchisq(model_1$deviance, model_1$df.residual, ncp=0, lower.tail = FALSE)
p_dev

# critical value
C_val <- qchisq(0.05, model_1$df.residual, ncp=0, lower.tail = FALSE)
C_val

#summary(model_1)
#pchisq(1168 - 44, df=(39-38))

anova(model_1,model_s, test = "LRT")   # saturated vs. final model



## Rao statistics

In [ ]:
##############################################################
######## Rao score statistics

Rao <- sum((Y-mu_est_1)^2/mu_est_1)
Rao
# p-hodnota testu adekvatnosti modelu (pomoci Raovy statistiky)
# H0: model dobre popisuje data
prao <- pchisq(Rao, model$df.residual, ncp=0, lower.tail = FALSE); 
prao

######  pomoci saturovaneho modelu
anova(model_1,model_s, test = "Rao")

Question: 

How to compare non-nested models?



...


# Your turn:
1. Generate data with followings parameters
 * $Y \sim Poi(\mu_i)$, where $E[Y_i] = \mu_i = \beta_0 + \beta_1 x_{i1} + \beta_2 x_{i2} = x_i^T \beta \  \Rightarrow \ q(\mu_i) = \mu_i =  x_i^T \beta  = \eta_i$
* $X_{i1} \sim N(50,10)$
* $X_{i2} \sim U(10,60)$
* $X_{i3} \sim Ber(0.45)$
* $n = 40$
2. Compute $\hat{\mu_i}$  for saturated, null,"full","best" models.
3. Compute Deviance, Rao, Wald statistics for your model and compare final model with the saturated and "full" ones.
4. Generate 100x data for  $n \in \{20,40,60,80,100 \}$ and plot $(\hat{\beta_i} -\beta_i)$ vs. $(n)$
